In [5]:
from dotenv import load_dotenv

load_dotenv()

import boto3
import os

s3 = boto3.resource('s3')
s3.Bucket(
    os.environ.get("BUCKET_NAME")
).download_file(
    os.environ.get("OBJECT_KEY"), 
    './downloads/file.csv',
)



AttributeError: 'S3' object has no attribute 'Bucket'